This is a fetching script so that I can fetch more cryptocurrency data.  
I use Bybit.

In [14]:
import os
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta

def fetch_crypto_data(symbol):
    api_key = os.getenv('BYBIT_API_KEY')
    api_secret = os.getenv('BYBIT_API_SECRET')
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
    })

    timeframe = '15m'
    rate_limit_sleep = 1 

    start_date = datetime.strptime('2023-12-19 00:00:00', '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime('2024-04-01 00:00:00', '%Y-%m-%d %H:%M:%S')

    all_bars = []

    current_time = start_date
    while current_time < end_date:
        try:
            since = int(current_time.timestamp() * 1000)
            bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=200)
            if not bars:
                break
            all_bars.extend(bars)
            last_time = bars[-1][0]
            current_time = datetime.utcfromtimestamp(last_time / 1000) + timedelta(hours=4)
            time.sleep(rate_limit_sleep)
        except ccxt.RateLimitExceeded as e:
            print("Rate limit exceeded")
            time.sleep(60)
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    historical_data = pd.DataFrame(all_bars, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    historical_data['datetime'] = pd.to_datetime(historical_data['datetime'], unit='ms')
    
    return historical_data

data = fetch_crypto_data('ADA/USDT')
data.to_csv("ADA_USDT_45m.csv", index=False)